# Parsing code 
This notebook is intended to show how the parsing of matpower, pss/e and plexos data can be done to integrate it with the MEMFS platform. 

Currently IPOPT is not working in Julia the [issue](https://github.com/JuliaOpt/Ipopt.jl/issues/61#issuecomment-339160928) has been raised and a solution is coming. While this gets resolved, the solution to powerflow models will use NLopt with the SQP solver. 

In [6]:
include("../parsers/parsers.jl")
using parsers
include("../ed_models/economic_dispatch.jl")
using economic_dispatch;
using PowerModels
using NLopt
using Ipopt
model_constructor = ACPPowerModel
solver1 = NLoptSolver(algorithm = :LD_SLSQP);
solver2 = IpoptSolver();

The plexos parser is working, but the sequences to parse load data from time series and the generation costs function is still missing. Information about the buses and branches has been already incorporated. 

In [2]:
plexos = plexoscsv_parser("../data_files/plexos_csv_118/")
data = matpower_parser("../data_files/case118.m")

Dict{String,Any} with 9 entries:
  "per_unit"     => true
  "name"         => "case118"
  "dcline"       => Dict{String,Any}()
  "baseMVA"      => 100
  "gen"          => Dict{String,Any}(Pair{String,Any}("32", Dict{String,Any}(Pa…
  "branch"       => Dict{String,Any}(Pair{String,Any}("1", Dict{String,Any}(Pai…
  "multinetwork" => false
  "version"      => "2"
  "bus"          => Dict{String,Any}(Pair{String,Any}("32", Dict{String,Any}(Pa…

Matpower files are parsed using PowerModels and the solution to the models has been modified to allow using power models pf solver with any type of datafile and not only MPC. 

In [4]:
plexos["gen"]["32"]

Dict{String,Any} with 30 entries:
  "start_up_cost" => 10.07
  "qc1max"        => 0.0
  "max_ramp_u"    => 0.42
  "pg"            => 0.0
  "model"         => 1
  "shutdown"      => 0.0
  "min_up_time"   => 6
  "startup"       => 10.07
  "qc2max"        => 0.0
  "fixed_cost"    => 6.91
  "qg"            => 0.0
  "gen_bus"       => 12
  "pmax"          => 1.9
  "min_dn_time"   => 6
  "time_series"   => Nullable{TimeSeries.TimeArray}
  "max_ramp_d"    => 0.42
  "mbase"         => 1.9
  "vg"            => 1.0
  "pc2"           => 0.0
  "dispatchable"  => true
  "index"         => 32
  "cost"          => Any[(12.12, 1.9)]
  "qmax"          => 9999.0
  "gen_status"    => 1
  "qmin"          => -9999.0
  ⋮               => ⋮

In [5]:
data["gen"]["32"]

Dict{String,Any} with 27 entries:
  "qc1max"     => 0.0
  "model"      => 2
  "startup"    => 0.0
  "qc2max"     => 0.0
  "qg"         => 0.0
  "gen_bus"    => 72
  "ramp_10"    => 0.0
  "mbase"      => 100.0
  "pc2"        => 0.0
  "index"      => 32
  "qmax"       => 1.0
  "pc1"        => 0.0
  "ramp_q"     => 0.0
  "ramp_30"    => 0.0
  "apf"        => 0.0
  "pg"         => 0.0
  "shutdown"   => 0.0
  "ramp_agc"   => 0.0
  "pmax"       => 1.0
  "vg"         => 0.98
  "cost"       => [100.0, 4000.0, 0.0]
  "gen_status" => 1
  "qmin"       => -1.0
  "qc1min"     => 0.0
  "qc2min"     => 0.0
  ⋮            => ⋮

In [5]:
#data = matpower_parser("../data_files/case118.m")
#pm_data = run_generic_model(data, model_constructor, solver2, PowerModels.post_pf)

The psse parse works properly and currently is only missing a way to create entries to the dictionary for 3-winding transformers. The test of building a power models model works, however, without the transformers the solver fails. 

Finally, the rutine ```dict_to_struct``` allows parsing the information in dictionaries into the type strucutre previously defined for the MEMFS platform. In this way, is possible to use both Powermodels and also MEMFS. 